In [1]:
%load_ext autoreload
%autoreload 2

# Compile mmcoremj.jar and copy dll and jar to Micro-Manager installation
import os
from pathlib import Path

os.chdir(r"C:\Users\henry\GitRepos\micro-manager\mmCoreAndDevices\MMCoreJ_wrap")
result = os.system("ant jar")
if result != 0:
    print(f"Error running 'ant jar': exit code {result}")

import shutil
source_1 = r"C:\Users\henry\GitRepos\micro-manager\mmCoreAndDevices\build\Debug\x64\MMCoreJ_wrap.dll"
destination_1 = r"C:\Program Files\Micro-Manager\MMCoreJ_wrap.dll"
shutil.copy2(source_1, destination_1)
source_2 = r"C:\Users\henry\GitRepos\micro-manager\build\Java\MMCoreJ.jar"
destination_2 = r"C:\Program Files\Micro-Manager\plugins\Micro-Manager\MMCoreJ.jar"
shutil.copy2(source_2, destination_2)
source_3 = r"C:\Users\henry\GitRepos\micro-manager\mmCoreAndDevices\build\Debug\x64\mmgr_dal_DemoCamera.dll"
destination_3 = r"C:\Program Files\Micro-Manager\mmgr_dal_DemoCamera.dll"
shutil.copy2(source_3, destination_3)



'C:\\Program Files\\Micro-Manager\\mmgr_dal_DemoCamera.dll'

In [2]:
%load_ext autoreload
%autoreload 2

JAVA_BACKEND = True

# import the buffer_test_util.py file
import importlib.util
import sys

# Import util file form absolute path
file_path = 'C:/Users/henry/GitRepos/mmpycorex/test/buffer_test_util.py'
spec = importlib.util.spec_from_file_location("buffer_test_util", file_path)
buffer_test_util = importlib.util.module_from_spec(spec)
sys.modules["buffer_test_util"] = buffer_test_util
spec.loader.exec_module(buffer_test_util)


from pycromanager import start_headless, stop_headless, Core
import time
import numpy as np
from buffer_test_util import *

if JAVA_BACKEND:
    start_headless('C:/Program Files/Micro-Manager', 'MMConfig_demo.cfg', max_memory_mb=9000)
else:
    start_headless('C:/Program Files/Micro-Manager', 'MMConfig_demo.cfg', python_backend=True)
    import pymmcore; print(pymmcore.__version__)

core = Core(convert_camel_case=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Test the ability for force reset the buffer
core.enableNewDataBuffer(True)

core.snapImage()
buffer_data_pointer = core.getImagePointer()
# no release
try:
    # this should raise an exception becuase of the unreleased pointer
    core.clearBuffer()
    assert False
except Exception as e:
    pass

core.forceBufferReset()
core.enableNewDataBuffer(True)
core.snapImage()
buffer_data_pointer = core.getImagePointer()
buffer_data_pointer.release()
core.clearBuffer()

In [6]:
# SnapImage Tests 

# Make sure we can get and release the pointer if v2 buffer is enabled
core.enableNewDataBuffer(True)

core.snapImage()
buffer_data_pointer = core.getImagePointer()
assert isinstance(buffer_data_pointer.getData(), np.ndarray)
buffer_data_pointer.release()
assert(buffer_data_pointer.getData() is None)


# Make sure we can't get the pointer if v2 buffer is disabled
core.enableNewDataBuffer(False)
core.snapImage()

try:
    core.getImagePointer()
    assert False
except Exception as e:
    pass


# Make sure regular snap image still works with either buffer
for v2 in [False, True]:
    core.enableNewDataBuffer(v2)
    core.snapImage()
    pixels = core.getImage()
    assert isinstance(pixels, np.ndarray)


if JAVA_BACKEND:
    ## Tagged image snap tests
    # Make sure regular snap image still works with either buffer
    for v2 in [False, True]:
        core.enableNewDataBuffer(v2)
        core.snapImage()
        tagged_image = core.getTaggedImage()
        assert isinstance(tagged_image.pix, np.ndarray)
        assert isinstance(tagged_image.tags, dict)


    # TaggedImagePointer 
    for v2 in [False, True]:
        core.enableNewDataBuffer(v2)
        core.snapImage()
        if not v2:
            # it should raise an exception if you try to get the pointer
            try:
                tagged_image = core.get_tagged_image_pointer()
                assert False
            except Exception as e:
                pass
        else:
            # it should return a pointer
            tagged_image = core.getTaggedImagePointer()
            assert tagged_image.pix is None
            tagged_image.getPixels()
            assert isinstance(tagged_image.pix, np.ndarray)


In [7]:
# Test functions for getting images and TaggedImages out of the buffer
from time import sleep, time
import numpy as np

if JAVA_BACKEND:
    # Test Peeking images/tagged images when running an overwrite mode sequence
    for v2 in [True, False]:
        core.enableNewDataBuffer(v2)
        core.setCircularBufferMemoryFootprint(10)
        core.startContinuousSequenceAcquisition(0)
        sleep(1)
        core.stopSequenceAcquisition()

        ti = core.getLastTaggedImage()
        assert isinstance(ti.pix, np.ndarray)
        i = core.getLastImage()
        assert isinstance(i, np.ndarray)
        # Test direct access to the BufferDataPointer object
        if v2:
            buffer_data_pointer = core.getLastDataPointer()
            assert isinstance(buffer_data_pointer.getData(), np.ndarray)
            buffer_data_pointer.release()
            assert(buffer_data_pointer.getData() is None)
            # Test the TaggedImagePointer object
            tip = core.getLastTaggedImagePointer()    
            assert tip.pix is None
            tip.getPixels()
            assert isinstance(tip.pix, np.ndarray)
            tip.release()
        else:
            # assert low level throws exception
            try:
                core.getLastDataPointer()
                assert False
            except Exception as e:
                pass
            # Assert high level gives none
            try:
                tip = core.getLastTaggedImagePointer()
                assert False
            except Exception as e:
                pass
        

    # Test popping tagged images when running a non-overwrite mode sequence
    for v2 in [True, False]:
        core.enableNewDataBuffer(v2)
        core.setCircularBufferMemoryFootprint(10)
        core.startSequenceAcquisition(4, 0, True)
        while core.isSequenceRunning():
            sleep(0.01)

        ti = core.popNextTaggedImage()
        assert isinstance(ti.pix, np.ndarray)
        i = core.popNextImage()
        assert isinstance(i, np.ndarray)
        # Test direct access to the BufferDataPointer object
        if v2:
            buffer_data_pointer = core.popNextDataPointer()
            assert isinstance(buffer_data_pointer.getData(), np.ndarray)
            buffer_data_pointer.release()
            assert(buffer_data_pointer.getData() is None)
            # Test the TaggedImagePointer object
            tip = core.popNextTaggedImagePointer()    
            assert tip.pix is None
            tip.getPixels()
            assert isinstance(tip.pix, np.ndarray)
            tip.release()
        else:
            # assert low level throws exception
            try:
                core.popNextDataPointer()
                assert False
            except Exception as e:
                pass
            # Assert high level gives none
            try:
                tip = core.popNextTaggedImagePointer()
                assert False
            except Exception as e:
                pass


In [8]:
# Test pulling out images from different cameras
from time import sleep


core.stopSequenceAcquisition('Camera')
core.stopSequenceAcquisition('Camera2')
image_size = 1024
image_size2 = 2048
buffer_size = 100

core.enableNewDataBuffer(True)
core.setBufferMemoryFootprint(buffer_size)

# change to differen image sizes and pixel types
core.setProperty('Camera', 'OnCameraCCDYSize', str(image_size))
core.setProperty('Camera', 'OnCameraCCDXSize', str(image_size))
core.setProperty('Camera', 'PixelType', '8bit')

core.setProperty('Camera2', 'OnCameraCCDYSize', str(image_size2))
core.setProperty('Camera2', 'OnCameraCCDXSize', str(image_size2))
core.setProperty('Camera2', 'PixelType', '16bit')

core.startContinuousSequenceAcquisition('Camera', 0)
core.startContinuousSequenceAcquisition('Camera2', 0)

sleep(0.5)

if JAVA_BACKEND:
    tip1 = core.getLastTaggedImagePointerFromDevice('Camera')
    tip2 = core.getLastTaggedImagePointerFromDevice('Camera2')

    assert isinstance(tip1.getPixels(), np.ndarray)
    assert isinstance(tip2.getPixels(), np.ndarray)

    assert tip1.tags.get('Camera') == 'Camera'
    assert tip2.tags.get('Camera') == 'Camera2'

    # check the data types are uint8 and uint16
    assert tip1.pix.dtype == np.uint8
    assert tip2.pix.dtype == np.uint16

    core.stopSequenceAcquisition('Camera')
    core.stopSequenceAcquisition('Camera2')
else:
    md1 = pymmcore.Metadata()
    md2 = pymmcore.Metadata()
    buffer_data_pointer1 = core.getLastDataFromDevicePointer('Camera')
    buffer_data_pointer2 = core.getLastDataFromDevicePointer('Camera2')

    buffer_data_pointer1.getMetadata(md1)
    buffer_data_pointer2.getMetadata(md2)

    assert md1.GetSingleTag('Camera').GetValue() == 'Camera'
    assert md2.GetSingleTag('Camera').GetValue() == 'Camera2'

    assert isinstance(buffer_data_pointer1.getData(), np.ndarray)
    assert isinstance(buffer_data_pointer2.getData(), np.ndarray)

    buffer_data_pointer1.release()
    buffer_data_pointer2.release()

    core.stopSequenceAcquisition('Camera')
    core.stopSequenceAcquisition('Camera2')



In [9]:
# Test whether you can wrap around more data greater than the buffer size
from time import sleep

num_images = 50
core.enableNewDataBuffer(True)
core.setExposure(25)

core.setCircularBufferMemoryFootprint(10)
core.startSequenceAcquisition(num_images, 0, True)

i = 0
while True:
    try:
        im = core.popNextImage()
        assert im is not None
        i += 1
        print(f'{i} popped: Free {core.getBufferFreeCapacity()}', f'Total {core.getBufferTotalCapacity()}')
    except:
        pass
    if core.isBufferOverflowed():
        print('overflowed, ', f'Free {core.getBufferFreeCapacity()}')
        break
    if not core.isSequenceRunning():
        print('sequence done')
    if i == num_images:
        break
    sleep(0.01)

1 popped: Free 10 Total 10
2 popped: Free 10 Total 10
3 popped: Free 10 Total 10
4 popped: Free 10 Total 10
5 popped: Free 10 Total 10
6 popped: Free 10 Total 10
7 popped: Free 10 Total 10
8 popped: Free 10 Total 10
9 popped: Free 10 Total 10
10 popped: Free 10 Total 10
11 popped: Free 10 Total 10
12 popped: Free 10 Total 10
13 popped: Free 10 Total 10
14 popped: Free 10 Total 10
15 popped: Free 10 Total 10
16 popped: Free 10 Total 10
17 popped: Free 10 Total 10
18 popped: Free 10 Total 10
19 popped: Free 10 Total 10
20 popped: Free 10 Total 10
21 popped: Free 10 Total 10
22 popped: Free 10 Total 10
23 popped: Free 10 Total 10
24 popped: Free 10 Total 10
25 popped: Free 10 Total 10
26 popped: Free 10 Total 10
27 popped: Free 10 Total 10
28 popped: Free 10 Total 10
29 popped: Free 10 Total 10
30 popped: Free 10 Total 10
31 popped: Free 10 Total 10
32 popped: Free 10 Total 10
33 popped: Free 10 Total 10
34 popped: Free 10 Total 10
35 popped: Free 10 Total 10
36 popped: Free 10 Total 10
3

In [10]:
# Test continuous acquisition (live mode) produces changing images
import numpy as np
from buffer_test_util import *

# Setup camera with desired parameters
image_size = 1024
exposure = 10
MB_per_image = setup_camera(core, image_size, exposure, fast_image=False)
buffer_size = 20
duration = 5.0

run_live_mode(core, image_size, use_new_data_buffer=True, buffer_size_mb=buffer_size, duration=duration, check_for_changing_images=True)
run_live_mode(core, image_size, use_new_data_buffer=False, buffer_size_mb=buffer_size, duration=duration, check_for_changing_images=True)


# Test live mode with RGB images
core.setProperty('Camera', 'PixelType', '32bitRGB')

MB_per_image = setup_camera(core, image_size, exposure, fast_image=False)

run_live_mode(core, image_size, use_new_data_buffer=True, buffer_size_mb=buffer_size, duration=duration, check_for_changing_images=True)
run_live_mode(core, image_size, use_new_data_buffer=False, buffer_size_mb=buffer_size, duration=duration, check_for_changing_images=True)


core.setProperty('Camera', 'PixelType', '16bit')


Continuous acquisition:   0%|          | 0/5.0 [00:00<?, ?it/s]

Buffer free:   0%|          | 0/20 [00:00<?, ?it/s]

Continuous acquisition:   0%|          | 0/5.0 [00:00<?, ?it/s]

Buffer free:   0%|          | 0/20 [00:00<?, ?it/s]

Continuous acquisition:   0%|          | 0/5.0 [00:00<?, ?it/s]

Buffer free:   0%|          | 0/5 [00:00<?, ?it/s]

Continuous acquisition:   0%|          | 0/5.0 [00:00<?, ?it/s]

Buffer free:   0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
# Run two cameras with different sizes
from time import sleep

num_images = 5
image_size = 1024
image_size2 = 2048
buffer_size = 500

core.enableNewDataBuffer(True)
core.setBufferMemoryFootprint(buffer_size)

# change to differen image sizes and pixel types
core.setProperty('Camera', 'OnCameraCCDYSize', str(image_size))
core.setProperty('Camera', 'OnCameraCCDXSize', str(image_size))
core.setProperty('Camera', 'PixelType', '8bit')

core.setProperty('Camera2', 'OnCameraCCDYSize', str(image_size2))
core.setProperty('Camera2', 'OnCameraCCDXSize', str(image_size2))
core.setProperty('Camera2', 'PixelType', '16bit')

core.startSequenceAcquisition('Camera', num_images, 0, True)
core.startSequenceAcquisition('Camera2', num_images, 0, True)

i = 0
images = []
while True:
    try:
        if JAVA_BACKEND:
            im = core.popNextTaggedImage()
            pix = im.pix
            tags = im.tags
        else:
            md = pymmcore.Metadata()
            pix = core.popNextImageMD(md)
        
        assert im is not None
        images.append((pix, tags if JAVA_BACKEND else md))
        i += 1
    except:
        pass
    if core.isBufferOverflowed():
        print('overflowed, ', f'Free {core.getBufferFreeCapacity()}')
        break
    if i == num_images * 2:
        break
    sleep(0.01)

# find the size of camera 1 and camera 2 in the tags and shape
# ensure that call camera 1 and camera 2 are the same
if JAVA_BACKEND:
    camera1_images = [im[0] for im in images if im[1]['Camera'] == 'Camera']
    camera2_images = [im[0] for im in images if im[1]['Camera'] == 'Camera2']
else:
    camera1_images = [im[0] for im in images if im[1].GetSingleTag('Camera').GetValue() == 'Camera']
    camera2_images = [im[0] for im in images if im[1].GetSingleTag('Camera').GetValue() == 'Camera2']

assert len(camera1_images) == len(camera2_images)
assert all(camera1_images[0][0].shape == im[0].shape for im in camera1_images)
assert all(camera2_images[0][0].shape == im[0].shape for im in camera2_images)    
# assert shapes are different
if JAVA_BACKEND:
    assert camera1_images[0].shape != camera2_images[0].shape
else:
    assert camera1_images[0][0].shape != camera2_images[0][0].shape
